In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import losses, layers, Model, optimizers
from tensorflow.keras import backend as K

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tensors = '/path/to/the/features/folder'

In [ ]:
clinical_train_features = np.load(os.path.join(tensors, 'features_univ_clinic_train.npy'))
clinical_val_features = np.load(os.path.join(tensors, 'features_univ_clinic_val.npy'))
clinical_test_features = np.load(os.path.join(tensors, 'features_univ_clinic_test.npy'))

print(clinical_train_features.shape, clinical_val_features.shape, clinical_test_features.shape)

In [ ]:
guided_clinical_train_features = np.load(os.path.join(tensors, 'features_clinic_guided_train.npy'))
guided_clinical_val_features = np.load(os.path.join(tensors, 'features_clinic_guided_val.npy'))
guided_clinical_test_features = np.load(os.path.join(tensors, 'features_clinic_guided_test.npy'))

print(guided_clinical_train_features.shape, guided_clinical_val_features.shape, guided_clinical_test_features.shape)

In [ ]:
train_labels = pd.read_csv('/path/to/label/folder/train_data_new.csv')
val_labels = pd.read_csv('/path/to/label/folder/val_data_new.csv')
test_labels = pd.read_csv('/path/to/label/folder/test_data_new.csv')

In [ ]:
label_dict_diagnosis = {'BCC':0, 'NEV':1, 'MEL':2, 'MISC':3, 'SK':4}
label_dict_pn = {'ABS':0, 'TYP':1, 'ATP':2}
label_dict_bwv = {'ABS':0, 'PRS':1}
label_dict_vs = {'ABS':0, 'REG':1, 'IR':2}
label_dict_pig = {'ABS':0, 'REG':1, 'IR':2}
label_dict_str = {'ABS':0, 'REG':1, 'IR':2}
label_dict_dag = {'ABS':0, 'REG':1, 'IR':2}
label_dict_rs = {'ABS':0, 'PRS':1}

In [ ]:
def get_trainvaltest_labels(task, label_dict, train_labels, val_labels, test_labels):
    y_train = []
    for label in train_labels[task].tolist():
        y_train.append(label_dict[label])

    y_val = []
    for label in val_labels[task].tolist():
        y_val.append(label_dict[label])

    y_test = []
    for label in test_labels[task].tolist():
        y_test.append(label_dict[label])
        
    return np.array(y_train), np.array(y_val), np.array(y_test)

In [ ]:
y_train_diagnosis, y_val_diagnosis, y_test_diagnosis = get_trainvaltest_labels('diagnosis', label_dict_diagnosis,
                                                                              train_labels, val_labels, test_labels)
y_train_pn, y_val_pn, y_test_pn = get_trainvaltest_labels('pigment_network', label_dict_pn, 
                                                          train_labels, val_labels, test_labels)
y_train_bwv, y_val_bwv, y_test_bwv = get_trainvaltest_labels('blue_whitish_veil', label_dict_bwv, 
                                                             train_labels, val_labels, test_labels)
y_train_vs, y_val_vs, y_test_vs = get_trainvaltest_labels('vascular_structures', label_dict_vs, 
                                                          train_labels, val_labels, test_labels)
y_train_pig, y_val_pig, y_test_pig = get_trainvaltest_labels('pigmentation', label_dict_pig,
                                                             train_labels, val_labels, test_labels)
y_train_str, y_val_str, y_test_str = get_trainvaltest_labels('streaks', label_dict_str, 
                                                             train_labels, val_labels, test_labels)
y_train_dag, y_val_dag, y_test_dag = get_trainvaltest_labels('dots_and_globules', label_dict_dag,
                                                                              train_labels, val_labels, test_labels)
y_train_rs, y_val_rs, y_test_rs = get_trainvaltest_labels('regression_structures', label_dict_rs, 
                                                          train_labels, val_labels, test_labels)

In [ ]:
class_weights_diagnosis = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_diagnosis), y=y_train_diagnosis)
class_weights_diagnosis_dict = dict(enumerate(class_weights_diagnosis))
print('diagnosis:', class_weights_diagnosis_dict)

class_weights_pn = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_pn), y=y_train_pn)
class_weights_pn_dict = dict(enumerate(class_weights_pn))
print('pn:', class_weights_pn_dict)

class_weights_bwv = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_bwv), y=y_train_bwv)
class_weights_bwv_dict = dict(enumerate(class_weights_bwv))
print('bwv:', class_weights_bwv_dict)

class_weights_vs = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_vs), y=y_train_vs)
class_weights_vs_dict = dict(enumerate(class_weights_vs))
print('vs:', class_weights_vs_dict)

class_weights_pig = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_pig), y=y_train_pig)
class_weights_pig_dict = dict(enumerate(class_weights_pig))
print('pig:', class_weights_pig_dict)

class_weights_str = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_str), y=y_train_str)
class_weights_str_dict = dict(enumerate(class_weights_str))
print('str:', class_weights_str_dict)

class_weights_dag = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_dag), y=y_train_dag)
class_weights_dag_dict = dict(enumerate(class_weights_dag))
print('dag:', class_weights_dag_dict)

class_weights_rs = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_rs), y=y_train_rs)
class_weights_rs_dict = dict(enumerate(class_weights_rs))
print('rs:', class_weights_rs_dict)

In [ ]:
# Concatenation

x_train = np.concatenate((clinical_train_features, guided_clinical_train_features), axis=1)
x_val = np.concatenate((clinical_val_features, guided_clinical_val_features), axis=1)
x_test = np.concatenate((clinical_test_features, guided_clinical_test_features), axis=1)

print(x_train.shape, x_val.shape, x_test.shape)

# Diagnosis

In [ ]:
print('Diagnosis')

print(x_train.shape, len(y_train_diagnosis))
print(x_val.shape, len(y_val_diagnosis))
print(x_test.shape, len(y_test_diagnosis))

In [ ]:
# Model diagnosis

input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_diagnosis = Model(inputs=input_layer, outputs=output_layer, name='model_diagnosis')
print(model_diagnosis.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_diagnosis.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_diagnosis.fit(x_train, y_train_diagnosis, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_diagnosis), verbose=0,
                        class_weight = class_weights_diagnosis_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_diagnosis=np.argmax(model_diagnosis.predict(x_test), axis=1)
balacc_diagnosis = metrics.balanced_accuracy_score(y_test_diagnosis, preds_diagnosis)

# Pigment Network

In [ ]:
# Model pn

input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_pn = Model(inputs=input_layer, outputs=output_layer, name='model_pn')
print(model_pn.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_pn.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_pn.fit(x_train, y_train_pn, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_pn), verbose=0,
                        class_weight = class_weights_pn_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_pn = np.argmax(model_pn.predict(x_test), axis=1)
balacc_pn = metrics.balanced_accuracy_score(y_test_pn, preds_pn)

# Blue Whitish Veil

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_bwv = Model(inputs=input_layer, outputs=output_layer, name='model_bwv')
print(model_bwv.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_bwv.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_bwv.fit(x_train, y_train_bwv, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_bwv), verbose=0,
                        class_weight = class_weights_bwv_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_bwv = np.argmax(model_bwv.predict(x_test), axis=1)
balacc_bwv = metrics.balanced_accuracy_score(y_test_bwv, preds_bwv)

# Vascular Structures

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_vs = Model(inputs=input_layer, outputs=output_layer, name='model_vs')
print(model_vs.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_vs.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_vs.fit(x_train, y_train_vs, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_vs), verbose=0,
                        class_weight = class_weights_vs_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_vs=np.argmax(model_vs.predict(x_test), axis=1)
balacc_vs = metrics.balanced_accuracy_score(y_test_vs, preds_vs)

# Pigmentation

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_pig = Model(inputs=input_layer, outputs=output_layer, name='model_pig')
print(model_pig.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_pig.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_pig.fit(x_train, y_train_pig, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_pig), verbose=0,
                        class_weight = class_weights_pig_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_pig = np.argmax(model_pig.predict(x_test), axis=1)
balacc_pig = metrics.balanced_accuracy_score(y_test_pig, preds_pig)

# Streaks

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_str = Model(inputs=input_layer, outputs=output_layer, name='model_str')
print(model_str.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_str.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

In [ ]:
m = model_str.fit(x_train, y_train_str, batch_size=100, epochs=1000, 
                        validation_data=(x_val, y_val_str), verbose=0,
                        class_weight = class_weights_str_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_str = np.argmax(model_str.predict(x_test), axis=1)
balacc_str = metrics.balanced_accuracy_score(y_test_str, preds_str)

# Dots and Globules

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_dag = Model(inputs=input_layer, outputs=output_layer, name='model_dag')
print(model_dag.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_dag.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_dag.fit(x_train, y_train_dag, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_dag), verbose=0,
                        class_weight = class_weights_dag_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_dag=np.argmax(model_dag.predict(x_test), axis=1)
balacc_dag = metrics.balanced_accuracy_score(y_test_dag, preds_dag)

# Regression Structures

In [ ]:
input_layer = layers.Input(shape=(4096,))
d1 = layers.Dropout(0.25)(input_layer)
layer2 = layers.Dense(100, activation='relu', name='mid')(d1)
d2 = layers.Dropout(0.25)(input_layer)
output_layer = layers.Dense(5, activation='softmax', name='output')(d2)

model_rs = Model(inputs=input_layer, outputs=output_layer, name='model_rs')
print(model_rs.summary())

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
scce = losses.SparseCategoricalCrossentropy()
model_rs.compile(loss=scce, optimizer=sgd)

earlystopping = tf.keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)

In [ ]:
m = model_rs.fit(x_train, y_train_rs, batch_size=100, epochs=500, 
                        validation_data=(x_val, y_val_rs), verbose=0,
                        class_weight = class_weights_rs_dict,
                        callbacks=[earlystopping])

In [ ]:
plt.plot(m.history['loss'])
plt.plot(m.history['val_loss'])
plt.title('title')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.grid()
plt.show()

In [ ]:
preds_rs = np.argmax(model_rs.predict(x_test), axis=1)
balacc_rs = metrics.balanced_accuracy_score(y_test_rs, preds_rs)